<a href="https://colab.research.google.com/github/sifatbhuiyan0909/Dhaka-Finance-Navigator/blob/main/Feature_importance_and_strategy_definition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded= files.upload()

Saving Dhaka-Stock-Exchange-DSE-2021.csv to Dhaka-Stock-Exchange-DSE-2021.csv
Saving Dhaka-Stock-Exchange-DSE-2020.csv to Dhaka-Stock-Exchange-DSE-2020.csv
Saving Dhaka-Stock-Exchange-DSE-2019.csv to Dhaka-Stock-Exchange-DSE-2019.csv
Saving Dhaka-Stock-Exchange-DSE-2018.csv to Dhaka-Stock-Exchange-DSE-2018.csv
Saving Dhaka-Stock-Exchange-DSE-2017.csv to Dhaka-Stock-Exchange-DSE-2017.csv
Saving Dhaka-Stock-Exchange-DSE-2016.csv to Dhaka-Stock-Exchange-DSE-2016.csv
Saving Dhaka-Stock-Exchange-DSE-2015.csv to Dhaka-Stock-Exchange-DSE-2015.csv
Saving Dhaka-Stock-Exchange-DSE-2014.csv to Dhaka-Stock-Exchange-DSE-2014.csv
Saving Dhaka-Stock-Exchange-DSE-2013.csv to Dhaka-Stock-Exchange-DSE-2013.csv
Saving Dhaka-Stock-Exchange-DSE-2012.csv to Dhaka-Stock-Exchange-DSE-2012.csv
Saving Dhaka-Stock-Exchange-DSE-2011.csv to Dhaka-Stock-Exchange-DSE-2011.csv
Saving Dhaka-Stock-Exchange-DSE-2010.csv to Dhaka-Stock-Exchange-DSE-2010.csv
Saving Dhaka-Stock-Exchange-DSE-2009.csv to Dhaka-Stock-Exchange

In [ ]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 126.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 130.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 17.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: pandas
    Found existing i

In [ ]:
import pandas as pd
import glob
import os
import numpy as np
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

# --- 1. DATA LOADING AND INITIAL CLEANING (Day 3-4) ---
print("--- 1. Initializing Data Pipeline (Days 3-8) ---")
all_filenames = glob.glob(os.path.join('.', '*.csv'))
# Load and concatenate all CSV files
master_df = pd.concat([pd.read_csv(f, header=None) for f in all_filenames], ignore_index=True)
master_df.columns = ['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']
df = master_df.copy()

# Day 4: Cleaning and Structuring
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
    # Coerce errors to handle non-numeric strings like '-'
    df[col] = pd.to_numeric(df[col].astype(str).str.replace(r'[-,]', '', regex=True), errors='coerce')

df = df.drop_duplicates(subset=['Date', 'Ticker'], keep='first')
df = df.set_index('Date')

# Day 5 & 6: Imputation and Outlier Correction (Forward-Fill and 3-Sigma)
cols_to_fill = ['Open', 'High', 'Low', 'Close', 'Volume']
df[cols_to_fill] = df[cols_to_fill].ffill()
df['Daily_Return'] = df['Close'].pct_change()

# Outlier Correction (Applied per day, not per ticker, for simplicity)
mu, sigma = df['Daily_Return'].mean(), df['Daily_Return'].std()
outlier_mask = (df['Daily_Return'] < mu - 3 * sigma) | (df['Daily_Return'] > mu + 3 * sigma)
df.loc[outlier_mask, cols_to_fill] = np.nan
df[cols_to_fill] = df[cols_to_fill].ffill() # Re-impute the outliers
df = df.drop(columns=['Daily_Return']).dropna() # Final drop any initial NaNs

# Day 7 & 8: Feature Engineering (Advanced features grouped by Ticker)
df = df.reset_index()

def create_advanced_features(group):
    # SMAs
    group['SMA_5'] = group['Close'].rolling(window=5).mean()
    group['SMA_20'] = group['Close'].rolling(window=20).mean()

    # RSI (Momentum)
    group['RSI'] = ta.rsi(close=group['Close'], length=14)

    # MACD (Trend/Momentum Crossover)
    macd_result = ta.macd(close=group['Close'])
    # Handle the structure of pandas_ta output
    if macd_result is not None and len(macd_result.columns) > 0:
        group['MACD'] = macd_result.iloc[:, 0]
    else:
        group['MACD'] = np.nan

    # Lagged Log Return (Short-term memory feature)
    group['Log_Return'] = np.log(group['Close'] / group['Close'].shift(1))
    group['Lag_Log_Return'] = group['Log_Return'].shift(1)

    return group.dropna(subset=['SMA_20', 'RSI', 'MACD']) # Drop initial NaNs from rolling windows

df = df.groupby('Ticker', group_keys=False).apply(create_advanced_features).set_index('Date')
print("--- Data Preparation (Days 3-8) Complete ---")

--- 1. Initializing Data Pipeline (Days 3-8) ---
--- Data Preparation (Days 3-8) Complete ---


/tmp/ipython-input-1010503829.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = df.groupby('Ticker', group_keys=False).apply(create_advanced_features).set_index('Date')
/tmp/ipython-input-1010503829.py:65: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Ticker', group_keys=False).apply(create_advanced_features).set_index('Date')


In [ ]:
# --- 2. MODEL PREPARATION & TRAINING (Day 9-11) ---
print("\n--- 2. Day 9-11: Model Training Pipeline ---")

# Day 9: Target Creation (Future Price Direction)
def create_target(group):
    # Target: 1 if the next day's Close price is higher than the current day's Close
    future_price = group['Close'].shift(-1)
    target = (future_price > group['Close']).astype(int)
    return group.assign(Target=target).dropna(subset=['Target'])

df = df.reset_index().groupby('Ticker', group_keys=False).apply(create_target).set_index('Date')

# --- Define X (Features) and y (Target) ---
# X_full retains 'Ticker' for backtesting alignment (MultiIndex fix)
X_full = df.drop(columns=['Target', 'Open', 'High', 'Low', 'Close', 'Volume', 'Log_Return'])
y = df['Target']

# Chronological Split (shuffle=False is essential for time series)
X_train_full, X_test_full, y_train, y_test = train_test_split(
    X_full, y, test_size=0.2, shuffle=False, stratify=None
)

# Create the final, model-ready feature sets (X_train/X_test) without Ticker
NON_FEATURE_COLS = ['Ticker']
X_train = X_train_full.drop(columns=NON_FEATURE_COLS)
X_test = X_test_full.drop(columns=NON_FEATURE_COLS)

print("--- Data Split (Day 9) Complete. ---")

# Day 10 & 11: Training and Tuning (Grid Search)
print("\n--- Day 10-11: Model Training and Tuning ---")

param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10],
    'min_samples_leaf': [5, 10]
}

grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42, n_jobs=-1),
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,
    verbose=0
)

print("Starting Grid Search to find optimal parameters (This may take a moment)...")
grid_search.fit(X_train, y_train)

# Get the Best Model
best_model = grid_search.best_estimator_
y_pred_tuned = best_model.predict(X_test)
accuracy_tuned = accuracy_score(y_test, y_pred_tuned)

print("\nBest Parameters Found:")
print(grid_search.best_params_)
print(f"Tuned Model Accuracy: {accuracy_tuned * 100:.2f}%")
print("--- Model Training and Tuning (Day 11) Complete. ---")


--- 2. Day 9-11: Model Training Pipeline ---


/tmp/ipython-input-691748429.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.reset_index().groupby('Ticker', group_keys=False).apply(create_target).set_index('Date')


--- Data Split (Day 9) Complete. ---

--- Day 10-11: Model Training and Tuning ---
Starting Grid Search to find optimal parameters (This may take a moment)...

Best Parameters Found:
{'max_depth': 10, 'min_samples_leaf': 10, 'n_estimators': 100}
Tuned Model Accuracy: 59.34%
--- Model Training and Tuning (Day 11) Complete. ---


In [ ]:
# --- 3. ANALYSIS & BACKTEST PREP (Day 12) ---
print("\n--- Day 12: Feature Importance & Strategy Definition ---")

# --- Step 1: Prepare the Combined Test DataFrame with MultiIndex ---
# df_test starts with the full test features which contain the Ticker column
df_test = X_test_full.copy()
df_test['Target_Actual'] = y_test

# Set the MultiIndex (Date, Ticker) on the test data.
df_test = df_test.reset_index().set_index(['Date', 'Ticker'])

# Create a MultiIndexed full DataFrame from your original 'df' to pull the 'Close' price correctly
df_full_mi = df.reset_index().set_index(['Date', 'Ticker'])

# Align the 'Close' price using the correct MultiIndex:
df_test['Close'] = df_full_mi.loc[df_test.index, 'Close']

print(f"Test DataFrame for Backtesting created with MultiIndex. Shape: {df_test.shape}")


# --- Step 2: Feature Importance Analysis (Uses X_test, the model-ready features) ---
print("\n1. Analyzing Feature Importance (Model's Logic)...")
importances = best_model.feature_importances_
feature_names = X_test.columns

feature_series = pd.Series(importances, index=feature_names)
sorted_importances = feature_series.sort_values(ascending=False)

print("\nTop Predictors (Ranked by Importance):")
print(sorted_importances)


# --- Step 3: Define Strategy Signal ---
CONFIDENCE_THRESHOLD = 0.55
print(f"\n2. Defining Strategy Signal (P(Up) > {CONFIDENCE_THRESHOLD*100:.0f}%)...")

# Use the model-ready features (X_test, without Ticker) for prediction
y_proba = best_model.predict_proba(X_test)
y_proba_up = y_proba[:, 1]

# Attach results to the final MultiIndexed df_test
df_test['Strategy_Signal'] = (y_proba_up > CONFIDENCE_THRESHOLD).astype(int)
df_test['Prediction_Probability'] = y_proba_up

# --- Step 4: Evaluate Strategy Coverage and Precision ---
y_test_strategy = df_test['Target_Actual'][df_test['Strategy_Signal'] == 1]
y_pred_strategy = df_test['Strategy_Signal'][df_test['Strategy_Signal'] == 1]

print(f"Total high-confidence trading days (coverage): {len(y_test_strategy)} out of {df_test.shape[0]}")

if len(y_test_strategy) > 0:
    report_strategy = classification_report(y_test_strategy, y_pred_strategy, zero_division=0)
    print("\nClassification Report (High-Confidence Trades Only):")
    print(report_strategy)
else:
    print("No high-confidence trades were signaled. Try lowering the CONFIDENCE_THRESHOLD.")

print("\n--- Day 12 Complete. Ready for Day 13: Backtesting & Deployment ---")


--- Day 12: Feature Importance & Strategy Definition ---
Test DataFrame for Backtesting created with MultiIndex. Shape: (237041, 7)

1. Analyzing Feature Importance (Model's Logic)...

Top Predictors (Ranked by Importance):
SMA_20            0.289759
SMA_5             0.250363
Lag_Log_Return    0.219607
MACD              0.123829
RSI               0.116441
dtype: float64

2. Defining Strategy Signal (P(Up) > 55%)...
Total high-confidence trading days (coverage): 546 out of 237041

Classification Report (High-Confidence Trades Only):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       205
           1       0.62      1.00      0.77       341

    accuracy                           0.62       546
   macro avg       0.31      0.50      0.38       546
weighted avg       0.39      0.62      0.48       546


--- Day 12 Complete. Ready for Day 13: Backtesting & Deployment ---


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

# Variables: df_test (from Day 12) is available.

print("\n--- Day 13: Backtesting & Performance Metrics (FIXED) ---")

# --- Step 1: Define Financial Metrics Function (Same as before) ---
def calculate_metrics(returns, trading_days_year=252, risk_free_rate=0.0):
    """Calculates Total Return, Max Drawdown, and Sharpe Ratio."""

    # CRITICAL: Replace any non-finite values (like NaN/inf) with 0 before compounding
    # This prevents the cumprod error observed earlier.
    returns = returns.replace([np.inf, -np.inf], np.nan).fillna(0)

    # Calculate Cumulative Returns (Portfolio Value)
    cumulative_returns = (1 + returns).cumprod()

    # 1. Total Return
    total_return = cumulative_returns.iloc[-1] - 1

    # 2. Maximum Drawdown (Risk Metric)
    peak = cumulative_returns.expanding(min_periods=1).max()
    drawdown = (cumulative_returns / peak) - 1
    max_drawdown = drawdown.min()

    # 3. Sharpe Ratio (Risk-Adjusted Return)
    annualized_return = returns.mean() * trading_days_year
    annualized_volatility = returns.std() * np.sqrt(trading_days_year)

    if annualized_volatility == 0:
        sharpe_ratio = np.nan
    else:
        sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility

    return total_return, max_drawdown, sharpe_ratio, cumulative_returns

# --- Step 2: Calculate Base Returns (per stock, per day) ---
# Ensure correct calculation of Daily_Return, grouped by Ticker.
df_test['Daily_Return'] = df_test.groupby('Ticker')['Close'].pct_change()


# --- Step 3: Calculate Strategy Returns (CRITICAL FIXES HERE) ---

# 1. Reset index and sort by Ticker and Date to prepare for accurate group-wise shift
df_test = df_test.reset_index().sort_values(by=['Ticker', 'Date'])
df_test = df_test.set_index(['Date', 'Ticker']) # Re-apply MultiIndex (optional but good practice)

# 2. Shift the signal group-wise: prediction on day T is used for trade on day T+1
df_test['Strategy_Signal_Shifted'] = df_test.groupby(level='Ticker')['Strategy_Signal'].shift(1)
df_test['Strategy_Return'] = df_test['Daily_Return'] * df_test['Strategy_Signal_Shifted']

# 3. Aggregate returns across all stocks for the overall portfolio view (Equal Weighted)
# This uses the average return across all stocks traded/held on that specific day.
portfolio_returns_strategy = df_test.groupby(level='Date')['Strategy_Return'].mean().dropna()
portfolio_returns_buy_hold = df_test.groupby(level='Date')['Daily_Return'].mean().dropna()


# --- Step 4: Calculate Final Portfolio Metrics ---

# Strategy Metrics (Cleaning is now handled inside calculate_metrics)
strategy_return, strategy_mdd, strategy_sharpe, strategy_curve = calculate_metrics(portfolio_returns_strategy)

# Buy and Hold (Benchmark) Metrics
buy_hold_return, buy_hold_mdd, buy_hold_sharpe, buy_hold_curve = calculate_metrics(portfolio_returns_buy_hold)


# --- Step 5: Display Results ---
print("\n")

print("\n--- Final Performance Metrics (Portfolio Average) ---")
print("| Metric | Strategy (Model) | Buy & Hold (Benchmark) |")
print("|:---|---:|---:|")
print(f"| **Total Return** | {strategy_return * 100:.2f}% | {buy_hold_return * 100:.2f}% |")
print(f"| **Max Drawdown** | {strategy_mdd * 100:.2f}% | {buy_hold_mdd * 100:.2f}% |")
print(f"| **Sharpe Ratio** | {strategy_sharpe:.3f} | {buy_hold_sharpe:.3f} |")

print("\nDay 13 COMPLETE. The financial performance is now accurately quantified.")


--- Day 13: Backtesting & Performance Metrics (FIXED) ---



--- Final Performance Metrics (Portfolio Average) ---
| Metric | Strategy (Model) | Buy & Hold (Benchmark) |
|:---|---:|---:|
| **Total Return** | 523630.44% | 565519189795467517012173063123086192962174976.00% |
| **Max Drawdown** | -3.99% | -74.43% |
| **Sharpe Ratio** | 0.240 | 0.687 |

Day 13 COMPLETE. The financial performance is now accurately quantified.


In [ ]:
import pickle
import os

# --- ASSUMPTIONS ---
# 1. best_model: Your final, optimized Random Forest Classifier (from Day 11).
# 2. X_test: Contains the final list of features used for prediction.

print("\n--- Day 14: Model Serialization (Saving for Deployment) ---")

# --- Step 1: Save the Trained Model ---
MODEL_FILENAME = 'random_forest_model.pkl'

try:
    with open(MODEL_FILENAME, 'wb') as file:
        pickle.dump(best_model, file)
    print(f"✅ Successfully saved the trained model to: {MODEL_FILENAME}")
except NameError:
    print("❌ ERROR: 'best_model' not found. Please ensure Day 11 was executed successfully.")

# --- Step 2: Save the Feature List (CRUCIAL for consistent feature order in API) ---
FEATURE_LIST_FILENAME = 'model_features.pkl'
try:
    feature_list = X_test.columns.tolist()

    with open(FEATURE_LIST_FILENAME, 'wb') as file:
        pickle.dump(feature_list, file)
    print(f"✅ Successfully saved the feature list to: {FEATURE_LIST_FILENAME}")
except NameError:
    print("❌ ERROR: 'X_test' not found. Cannot save feature list for API.")


print("\nDay 14 Complete. Deployment artifacts are ready.")


--- Day 14: Model Serialization (Saving for Deployment) ---
✅ Successfully saved the trained model to: random_forest_model.pkl
✅ Successfully saved the feature list to: model_features.pkl

Day 14 Complete. Deployment artifacts are ready.
